HIWRAP 3d tile viz

In [1]:

import os
import zarr
import numpy as np
import xarray as xr
import shutil
import boto3
from pathlib import Path

UTILS

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import glob, os, io
import gzip
import boto3, s3fs
from boto3 import client as boto_client
from datetime import date, time, datetime, timedelta

def down_vector(roll, pitch, head):
    x = np.sin(roll) * np.cos(head) + np.cos(roll) * np.sin(pitch) * np.sin(head)
    y = -np.sin(roll) * np.sin(head) + np.cos(roll) * np.sin(pitch) * np.cos(head)
    z = -np.cos(roll) * np.cos(pitch)
    return (x, y, z)


def add24hr(hr):
    """Correction of time in CRS for going over the next day in UTC"""
    b = np.where(hr < hr[0])
    hr[b] = hr[b] + 24
    return hr


def CRSaccess(fname, s3bucket=False, Verb=False):
    """
    Access the CRS file
    Return CRS filename with path (absolute path) for "local" access
    Return CRS data as object for "cloud access"
    Either way, the return value can be open by Xarray as netcdf file object
    """

    print("\%% Accessing data from Cloud. This may take a little time...\n")
    s3 = boto_client('s3')
    fileobj = s3.get_object(Bucket=s3bucket, Key=fname)
    fileCRS = fileobj['Body'].read()
    

def upload_to_s3(file_name, bucket, s3_name=None):
    """Upload a file to an S3 bucket
     file_name: File to upload
     bucket: S3 bucket to upload to
     object_name: S3 object name. If not specified then file_name is used
    """
    if s3_name is None: s3_name = file_name

    s3 = boto3.client('s3')
    try:
        s3.upload_file(file_name, bucket, s3_name)
    except ClientError as e:
       print(e)
    except NoCredentialsError:
        print("%%Credentials not available")



In [3]:
import os
import json
import numpy as np
from datetime import datetime
from threading import Thread, Lock

to_rad = np.pi / 180.0
to_deg = 180.0 / np.pi

steps = [32, 16, 8, 4, 2, 1]

class PointCloud:
    def __init__(self, key, lon, lat, alt, value, time, epoch):
        self.key = key
        self.lon = lon
        self.lat = lat
        self.alt = alt
        self.time = time
        self.value = value
        self.epoch = epoch
        self.tasks = []
        self.threads = []
        for i in range(10):
            self.threads.append(Thread(target=self.worker_function))
        self.tileset_lock = Lock()
        self.tileset_json = {
        	"asset": {
        		"version": "1.0",
        		"type": "Airborne Radar"
        	},
        	"root": {
        		"geometricError": 1000000,
        		"refine" : "REPLACE",
        		"boundingVolume": {
                    "region": [
                        float(np.min(lon)) * to_rad,
                        float(np.min(lat)) * to_rad,
                        float(np.max(lon)) * to_rad,
                        float(np.max(lat)) * to_rad,
                        float(np.min(alt)) * to_rad,
                        float(np.max(alt)) * to_rad
                    ]
                },
                "children": []
        	},
            "properties": {
                "epoch": "{}Z".format(datetime.utcfromtimestamp(epoch).isoformat()),
                "refined": []
            }
        }


    def worker_function(self):
        while len(self.tasks) > 0:
                tile, start, end = self.tasks.pop()
                print(tile, start, end)
                self.generate(tile, start, end)


    def start(self):
        for t in self.threads:
            t.start()


    def join(self):
        for t in self.threads:
            t.join()
        with open('{}/tileset.json'.format(self.key), mode='w+') as outfile:
            json.dump(self.tileset_json, outfile)


    def schedule_task(self, tile, start, end):
        self.tasks.append((tile, start, end))


    def generate(self, tile, start, end):
        print(tile, start, end)
        parent_tile = self.tileset_json["root"]
        cartesian, offset, scale, cartographic, region = self.cartographic_to_cartesian(start, end)

        value = self.value[start:end]
        time = self.time[start:end]

        epoch = int(np.min(time) + self.epoch - 300)
        epoch = "{}Z".format(datetime.utcfromtimestamp(epoch).isoformat())
        end = int(np.max(time) + self.epoch + 300)
        end = "{}Z".format(datetime.utcfromtimestamp(end).isoformat())

        header_length = 28
        magic = np.string_("pnts")
        version = 1

        for step in steps:
            self.tileset_lock.acquire()
            try:
                filename = "{}_{}.pnts".format(tile, step)
                child_tile = {
                    "availability": "{}/{}".format(epoch, end),
                    "geometricError": step * 500,
                    "boundingVolume": {
                        "region": region
                    },
                    "content": {
                        "uri": filename
                    },
                    "refine": "REPLACE"
                }
                if step == 1:
                    self.tileset_json["properties"]["refined"].append(filename)
                else:
                    child_tile["children"] = []
                parent_tile["children"].append(child_tile)
                parent_tile = child_tile
            finally:
                self.tileset_lock.release()

            tile_length = 0
            feature_table_binary_byte_length = 0
            batch_table_binary_byte_length = 0
            length = value[::step].size

            feature_table_json = {
                "POINTS_LENGTH": length,
                "BATCH_LENGTH": length,
                "BATCH_ID": {
                    "byteOffset": 0,
                    "componentType": "UNSIGNED_INT"
                },
                "POSITION_QUANTIZED": {
                    "byteOffset": length * 4
                },
                "QUANTIZED_VOLUME_OFFSET": offset,
                "QUANTIZED_VOLUME_SCALE": scale
            }

            batch_table_json = {
                "value": {
                    "byteOffset": 0,
                    "componentType": "FLOAT",
                    "type": "SCALAR"
                },
                "time": {
                    "byteOffset": length * 4,
                    "componentType": "FLOAT",
                    "type": "SCALAR"
                },
                "location": {
                    "byteOffset": length * 8,
                    "componentType": "SHORT",
                    "type": "VEC3"
                }
            }

            tile_length += header_length

            feature_table_json_min = json.dumps(feature_table_json, separators=(",", ":")) + "       "
            feature_table_trim = (tile_length + len(feature_table_json_min)) % 8
            if feature_table_trim != 0:
                feature_table_json_min = feature_table_json_min[:-feature_table_trim]

            tile_length += len(feature_table_json_min)

            feature_table_binary_byte_length = length * 4 + length * 3 * 2
            tile_length += feature_table_binary_byte_length
            feature_table_padding = tile_length % 8
            if feature_table_padding != 0:
                feature_table_padding = 8 - feature_table_padding
            tile_length += feature_table_padding

            batch_table_json_min = json.dumps(batch_table_json, separators=(",", ":")) + "       "
            batch_table_trim = (tile_length + len(batch_table_json_min)) % 8
            if batch_table_trim != 0:
                batch_table_json_min = batch_table_json_min[:-batch_table_trim]

            tile_length += len(batch_table_json_min)

            batch_table_binary_byte_length = length * 4 * 2 + length * 2 * 3
            tile_length += batch_table_binary_byte_length
            batch_table_padding = tile_length % 8
            if batch_table_padding != 0:
                batch_table_padding = 8 - batch_table_padding
            tile_length += batch_table_padding

            with open('{}/{}'.format(self.key, filename), mode='wb+') as outfile:
                outfile.write(np.string_(magic).tobytes())
                outfile.write(np.uint32(version).tobytes())
                outfile.write(np.uint32(tile_length).tobytes())
                outfile.write(np.uint32(len(feature_table_json_min)).tobytes())
                outfile.write(np.uint32(feature_table_binary_byte_length + feature_table_padding).tobytes())
                outfile.write(np.uint32(len(batch_table_json_min)).tobytes())
                outfile.write(np.uint32(batch_table_binary_byte_length + batch_table_padding).tobytes())
                outfile.write(np.string_(feature_table_json_min).tobytes())
                outfile.write(np.arange(length, dtype=np.uint32).tobytes())
                outfile.write(cartesian[::step, :].tobytes())
                for _ in range(feature_table_padding):
                    outfile.write(np.string_(" ").tobytes())
                outfile.write(np.string_(batch_table_json_min).tobytes())
                outfile.write(value[::step].astype(np.float32).tobytes())
                outfile.write(time[::step].astype(np.float32).tobytes())
                outfile.write(cartographic[::step, :].tobytes())
                for _ in range(batch_table_padding):
                    outfile.write(np.string_(" ").tobytes())
                outfile.seek(0)


    def cartographic_to_cartesian(self, start, end):
        lon = self.lon[start:end]
        lat = self.lat[start:end]
        alt = self.alt[start:end]
        size = lon.size

        cartographic = np.zeros(shape=(size, 3), dtype=np.int16)
        cartographic[:, 0] = (lon * 32767 / 180).astype(np.int16)
        cartographic[:, 1] = (lat * 32767 / 180).astype(np.int16)
        cartographic[:, 2] = (alt / 10).astype(np.int16)

        lon = lon * to_rad
        lat = lat * to_rad

        radiiSquared = np.array([40680631590769, 40680631590769, 40408299984661.445], dtype=np.float64)

        N1 = np.multiply(np.cos(lat), np.cos(lon))
        N2 = np.multiply(np.cos(lat), np.sin(lon))
        N3 = np.sin(lat)

        magnitude = np.sqrt(np.square(N1) + np.square(N2) + np.square(N3))

        N1 = N1 / magnitude
        N2 = N2 / magnitude
        N3 = N3 / magnitude

        K1 = radiiSquared[0] * N1
        K2 = radiiSquared[1] * N2
        K3 = radiiSquared[2] * N3

        gamma = np.sqrt(np.multiply(N1, K1) + np.multiply(N2, K2) + np.multiply(N3, K3))

        K1 = K1 / gamma
        K2 = K2 / gamma
        K3 = K3 / gamma

        N1 = np.multiply(N1, alt)
        N2 = np.multiply(N2, alt)
        N3 = np.multiply(N3, alt)

        # x = np.multiply((N1 + K1), np.random.normal(1, .00005, N1.size))
        # y = np.multiply((N2 + K2), np.random.normal(1, .00005, N1.size))
        # z = np.multiply((N3 + K3), np.random.normal(1, .00005, N1.size))

        x = N1 + K1
        y = N2 + K2
        z = N3 + K3

        offset = [float(np.min(x)), float(np.min(y)), float(np.min(z))]

        x = x - offset[0]
        y = y - offset[1]
        z = z - offset[2]

        scale = [float(abs(np.max(x))), float(abs(np.max(y))), float(abs(np.max(z)))]

        cartesian = np.zeros(shape=(size, 3), dtype=np.uint16)
        cartesian[:, 0] = (x / scale[0] * 65535.0).astype(np.uint16)
        cartesian[:, 1] = (y / scale[1] * 65535.0).astype(np.uint16)
        cartesian[:, 2] = (z / scale[2] * 65535.0).astype(np.uint16)

        region = [
            float(np.min(lon)),
            float(np.min(lat)),
            float(np.max(lon)),
            float(np.max(lat)),
            float(np.min(alt)),
            float(np.max(alt))
        ]

        return cartesian, offset, scale, cartographic, region


In [4]:
import sys
import os
import zarr
import numpy as np
import json
import datetime as dt

to_rad = np.pi / 180.0
to_deg = 180.0 / np.pi

def generate_point_cloud(variable, epoch, end, zarr_location, point_cloud_folder):
    """Generates json pointcloud from a given zarr file input

    Args:
        variable (_type_): _description_
        epoch (_type_): _description_
        end (_type_): _description_
        zarr_location (string): source zarr file.
        point_cloud_folder (string): destination folder for 3d tile json file.
    """

    #out_key = f"{os.getenv('CRS_OUTPUT_FLIGHT_PATH')}/{shortname}"
    #pc_out_key = f"{output_path}/point_cloud"

    '''
    try:
        os.mkdir(out_key)
    except:
        pass
    '''

    try:
        os.mkdir(point_cloud_folder)
    except:
        pass

    # LOAD THE DATA.
    store = zarr.DirectoryStore(zarr_location)
    root = zarr.group(store=store)

    chunk_id = root["chunk_id"][:]
    num_chunks = chunk_id.shape[0]
    id = np.argmax(chunk_id[:, 1] > epoch) - 1
    start_id = chunk_id[0 if id < 0 else id, 0]
    id = num_chunks - np.argmax(chunk_id[::-1, 1] < end)
    end_id = chunk_id[id, 0] if id < num_chunks else root["time"].size - 1

    root_epoch = root.attrs["epoch"]
    location = root["location"][start_id:end_id]
    lon = location[:, 0]
    lat = location[:, 1]
    alt = location[:, 2]
    value = root["value"][variable][start_id:end_id]
    time = root["time"][start_id:end_id]

    # filter data using mask
    epoch = epoch - root_epoch # date-time
    end = end - root_epoch
    mask = np.logical_and(time >= epoch, time <= end)
    lon = lon[mask]
    lat = lat[mask]
    alt = alt[mask]
    value = value[mask]
    time = time[mask]

    # Generate Pointcloud Tileset
    point_cloud = PointCloud(point_cloud_folder, lon, lat, alt, value, time, root_epoch)

    for tile in range(int(np.ceil(time.size / 530000))):
        start_id = tile * 530000
        end_id = np.min([start_id + 530000, time.size])
        point_cloud.schedule_task(tile, start_id, end_id)

    point_cloud.start()
    point_cloud.join()

tileset_json = {
	"asset": {
		"version": "1.0",
		"type": "Airborne Radar"
	},
	"root": {
		"geometricError": 1000000,
		"refine" : "REPLACE",
		"boundingVolume": {
            "region": []
        },
        "children": []
	},
    "properties": {
        "epoch": "",
        "refined": []
    }
}

#if __name__ == '__main__':
#    main(sys.argv[1], sys.argv[2], int(sys.argv[3]), int(sys.argv[4]))


In [5]:
# META needed for ingest
campaign = 'Hs3'
collection = "AirborneRadar"
dataset = "gpmValidationOlympexHiwrap"
variables = ["ref"]
renderers = ["point_cloud"]
chunk = 262144
to_rad = np.pi / 180
to_deg = 180 / np.pi

NOTE: for hiwrap data the direct s3access as file object didnot work. Maybe due to large size
need to point to the local data file for the ingest to work.

In [22]:
# fileObj = "../../../../tmp/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc"
fileObj = "/tmp/Hs3/hiwrap/zarr/20130925/nc/HS3_HIWRAP_20130925_kuouterchirp_175902-183052_v03.nc"

# open dataset.
with xr.open_dataset(fileObj, decode_cf=False) as ds:
   
    # data columns extract
    ref = ds[variables[0]].values #Hiwrap radar reflectivity
    lat = ds['lat'].values
    lon = ds['lon'].values
    alt = ds['alt'].values # altitude of aircraft in meters
    roll = ds["roll"].values
    pitch = ds["pitch"].values
    head = ds["head"].values
    rad_range = ds["range"].values
num_col = ref.shape[0] # number of cols
num_row = ref.shape[1] # number of rows

OSError: [Errno -101] NetCDF: HDF error: b'/tmp/Hs3/hiwrap/zarr/20130925/nc/HS3_HIWRAP_20130925_kuouterchirp_175902-183052_v03.nc'

In [10]:
ds

<xarray.Dataset>
Dimensions:       (time: 115064, range: 353)
Coordinates:
  * range         (range) float32 74.95 149.9 224.8 ... 2.638e+04 2.646e+04
  * time          (time) float32 3.239e+05 3.239e+05 ... 3.259e+05 3.259e+05
Data variables: (12/34)
    alt           (time) float32 1.673e+04 1.673e+04 ... 1.68e+04 1.68e+04
    dopcorr       (time, range) float32 ...
    doph          (time, range) float32 ...
    dophu         (time, range) float32 ...
    dopl          (time, range) float32 ...
    doplh         (time, range) float32 ...
    ...            ...
    vacft         (time) float32 ...
    vuh           float32 ...
    vul           float32 ...
    vulh          float32 ...
    wvel          (time) float32 ...
    year          int16 ...
Attributes:
    title:        HIWRAP Data, NASA Goddard Space Flight Center
    filename:     /karldata4/tian/HS3/netcdf/2013/dist/dat/v3/hs3_hiwrap_kain...
    experiment:   NASA HS3
    source:       created from HIWRAP binary file, Version 0.0
    comments:     Contact: lin.tian-1@nasa.gov
    Conventions:  CF-1.6
    history:      Wed Sep 23 15:20:50 2015: ncks -4 -L 4 -O /ghrc/ops/output/...
    NCO:          4.3.7

In [18]:
seconds_array = ds['time'].values
seconds_array
# date = "20130925"
# base_date = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))

# # Convert seconds to timedelta
timedelta_array = seconds_array.astype('timedelta64[s]')

# # Add timedelta to base date to get datetime array
datetime_array = np.array([base_date + delta for delta in timedelta_array])
datetime_array

array(['2013-09-28T17:59:05', '2013-09-28T17:59:05',
       '2013-09-28T17:59:05', ..., '2013-09-28T18:31:39',
       '2013-09-28T18:31:39', '2013-09-28T18:31:39'],
      dtype='datetime64[s]')

In [7]:

def ingest(folder, file, s3bucket):
    """
    Converts Level 1B hiwrap data from s3 to zarr file and then stores it in the provided folder
    Args:
        folder (string): name to hold the raw files.
        file (string): the s3 url to the raw file.
    """
    store = zarr.DirectoryStore(folder)
    root = zarr.group(store=store)
    
    # Create empty rows for modified data    
    z_chunk_id = root.create_dataset('chunk_id', shape=(0, 2), chunks=None, dtype=np.int64)
    z_location = root.create_dataset('location', shape=(0, 3), chunks=(chunk, None), dtype=np.float32)
    z_time = root.create_dataset('time', shape=(0), chunks=(chunk), dtype=np.int32)
    z_vars = root.create_group('value')
    z_ref = z_vars.create_dataset('ref', shape=(0), chunks=(chunk), dtype=np.float32)
    n_time = np.array([], dtype=np.int64)

    date = file.split("_")[2]
    base_time = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))

    print("Accessing file from S3 ", file)

    # read from s3 url (file) in s3 bucket.
    # fileObj = CRSaccess(file, s3bucket=s3bucket)
    fileObj = "../../../tmp/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc"

    # open dataset.
    with xr.open_dataset(fileObj, decode_cf=False) as ds:
        # added for time correction for over 24h UTC
        hr = add24hr(ds['time'].values)
        delta = (hr * 3600).astype('timedelta64[s]') + base_time
        # time correction end
        
        # data columns extract
        ref = ds[variables[0]].values #Hiwrap radar reflectivity
        lat = ds['lat'].values
        lon = ds['lon'].values
        alt = ds['alt'].values # altitude of aircraft in meters
        roll = ds["roll"].values
        pitch = ds["pitch"].values
        head = ds["head"].values
        rad_range = ds["range"].values
    num_col = ref.shape[0] # number of cols
    num_row = ref.shape[1] # number of rows

    # data frame formation
    delta = np.repeat(delta, num_row)
    lon = np.repeat(lon, num_row)
    lat = np.repeat(lat, num_row)
    alt = np.repeat(alt, num_row)
    roll = np.repeat(roll * to_rad, num_row)
    pitch = np.repeat(pitch * to_rad, num_row)
    head = np.repeat(head * to_rad, num_row)
    rad_range = np.tile(rad_range, num_col)
    ref = ref.flatten()

    # time correction.
    time = (delta - np.datetime64('1970-01-01')).astype('timedelta64[s]').astype(np.int64)

    x, y, z = down_vector(roll, pitch, head)
    x = np.multiply(x, np.divide(rad_range, 111000 * np.cos(lat * to_rad)))
    y = np.multiply(y, np.divide(rad_range, 111000))
    z = np.multiply(z, rad_range)

    lon = np.add(-x, lon)
    lat = np.add(-y, lat)
    alt = np.add(z, alt)

    # sort data by time
    sort_idx = np.argsort(time)

    lon = lon[sort_idx]
    lat = lat[sort_idx]
    alt = alt[sort_idx]
    ref = ref[sort_idx]
    time = time[sort_idx]

    # remove nan and infinite using mask ???
    mask = np.logical_and(np.isfinite(ref), alt > 0)
    lon = lon[mask]
    lat = lat[mask]
    alt = alt[mask]
    ref = ref[mask]
    time = time[mask]

    # Now populate (append) the empty rows with modified data.
    z_location.append(np.stack([lon, lat, alt], axis=-1))
    z_ref.append(ref)
    n_time = np.append(n_time, time)

    idx = np.arange(0, n_time.size, chunk)
    chunks = np.zeros(shape=(idx.size, 2), dtype=np.int64)
    chunks[:, 0] = idx
    chunks[:, 1] = n_time[idx]
    z_chunk_id.append(chunks)

    epoch = np.min(n_time)
    n_time = (n_time - epoch).astype(np.int32)
    z_time.append(n_time)

    # save it.
    root.attrs.put({
        "campaign": campaign,
        "collection": collection,
        "dataset": dataset,
        "variables": variables,
        "renderers": renderers,
        "epoch": int(epoch)
    })


Trigger point


In [8]:

def data_pre_process(bucket_name, field_campaign, input_data_dir, output_data_dir, instrument_name):
    s3_resource = boto3.resource('s3')
    s3bucket = s3_resource.Bucket(bucket_name)    
    keys = []
    for obj in s3bucket.objects.filter(
            Prefix=f"{field_campaign}/{input_data_dir}/{instrument_name}/HS3_HIWRAP_"):
        keys.append(obj.key)

    result = keys
    for s3_raw_file_key in result:
        # SOURCE DIR.
        sdate = s3_raw_file_key.split('_')[2]
        print(s3_raw_file_key)        
        print(sdate)        
        # return
        
        print(f'processing CRS file {s3_raw_file_key}')

        # CREATE A LOCAL DIR TO HOLD RAW DATA AND CONVERTED DATA
        folder = f"/tmp/hiwrap_hs3/zarr/{sdate}"
        point_cloud_folder = f"{folder}/point_cloud"
        if os.path.exists(folder): shutil.rmtree(f"{folder}")
        # os.mkdir(folder)
        Path(folder).mkdir(parents=True, exist_ok=True)
        # LOAD FROM SOURCE WITH NECESSARY PRE PROCESSING. CONVERT LEVEL 1B RAW FILES INTO ZARR FILE.
        ingest(folder, s3_raw_file_key, bucket_name)
        # CONVERT ZARR FILE INTO 3D TILESET JSON.
        generate_point_cloud("ref",  0,  1000000000000, folder, point_cloud_folder)

        # UPLOAD CONVERTED FILES.
        files = os.listdir(point_cloud_folder)
        s3 = boto3.client('s3')
        for file in files:
            fname = os.path.join(point_cloud_folder, file) # SOURCE
            s3name = f"{field_campaign}/{output_data_dir}/hiwrap/{sdate}/{file}" # DESTINATION
            print(f"uploaded to {s3name}.")
            upload_to_s3(fname, bucket_name, s3_name=s3name)


def hiwrap():
    # bucket_name = os.getenv('RAW_DATA_BUCKET')
    bucket_name="ghrc-fcx-field-campaigns-szg"
    field_campaign = "Hs3"
    input_data_dir = "instrument-raw-data"
    output_data_dir = "instrument-processed-data"
    instrument_name = "hiwrap"
    data_pre_process(bucket_name, field_campaign, input_data_dir, output_data_dir, instrument_name)


hiwrap()

Hs3/instrument-raw-data/hiwrap/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc
20130925
processing CRS file Hs3/instrument-raw-data/hiwrap/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc
Accessing file from S3  Hs3/instrument-raw-data/hiwrap/HS3_HIWRAP_20130925_kainnerchirp_175905-183056_v03.nc
49 25970000 26055482
49 25970000 26055482
48 25440000 25970000
48 25440000 25970000
47 24910000 25440000
47 24910000 25440000
46 24380000 24910000
46 24380000 24910000
45 23850000 24380000
45 23850000 24380000
44 23320000 23850000
44 23320000 23850000
43 22790000 23320000
43 22790000 23320000
42 22260000 22790000
42 22260000 22790000
41 21730000 22260000
41 21730000 22260000
40 21200000 21730000
40 21200000 21730000
39 20670000 21200000
39 20670000 21200000
38 20140000 20670000
38 20140000 20670000
37 19610000 20140000
37 19610000 20140000
36 19080000 19610000
36 19080000 19610000
35 18550000 19080000
35 18550000 19080000
34 18020000 18550000
34 18020000 18550000
33 17490000 18020000